In [1]:
import re

def get_floats(string):
    """
    Gibt alle Kommazahlen des übergebenen Strings als float-Liste zurück.
    
    Eingabebeispiel:
    string = POINT (51.375887 7.660134)
    
    Ausgabebeispiel:
    [51.375887, 7.660134]
    
    """
    
    return [float(i) for i in re.findall("\d+\.\d+", string)]


In [2]:
from math import sin, cos, sqrt, atan2, radians, pi

def calculate_distance(orig_point, dest_point):
    """
    Gibt den Abstand zwischen den beiden übergebenen Punkten in Kilometern als float zurück.
    
    Eingabebeispiel:
    orig_point = (51.375887, 7.660134)
    dest_point = (51.380709, 7.671346)
    
    Ausgabebeispiel:
    0.9450082401845116
    
    """
    
    # Erdradius
    R = 6371.0
    
    # Umrechnung von Gradmaß in Bogenmaß
    lat1 = radians(orig_point[0])
    lon1 = radians(orig_point[1])
    lat2 = radians(dest_point[0])
    lon2 = radians(dest_point[1])
    
    # Berechnung der Distanz
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c


def calculate_center(actions):
    """
    Gibt den zentralen Punkt der übergebenen Punkte als Liste zurück.
    Dieser Punkt repräsentiert den Durchschnitt der durch das Tracking-System gesammelten Punkte.
    
    Eingabebeispiel:
    actions = [[51.37459, 7.697596], [51.374501, 7.697512], [51.374204, 7.69759]]
    
    Ausgabebeispiel:
    [51.37443166667289, 7.6975660000324595]
    
    """
    
    x = y = z = 0
    for action in actions:
        # Umrechnung von Gradmaß in Bogenmaß
        lat = radians(action[0])
        lon = radians(action[1])
        # Summe der kartesischen Koordianten bilden
        x += cos(lat) * cos(lon)
        y += cos(lat) * sin(lon)
        z += sin(lat)

    #Durchschnitt der kartesischen Koordianten bilden
    x /= len(actions)
    y /= len(actions)
    z /= len(actions)

    #Umrechnung der kartesischen Koordianten in Bogenmaß und anschließend zurück in Gradmaß (Breiten- und Längengrade)
    lon = atan2(y, x) * 180/pi
    hyp = sqrt(x * x + y * y)
    lat = atan2(z, hyp) * 180/pi
    
    return [lat, lon]
    

In [3]:
from collections import defaultdict

def get_actions(cursor):
    """
    Gibt die Actions aus der Datenbank als Listen-Dictionary zurück.
    Wenn mehrere Actions (Tracking-Informationen) für einen Behälter vorliegen, wird das Zentrum dieser Punkte ermittelt.
    Dieses Zentrum repräsentiert alle Actions für diesen Behälter.
    
    Eingabe: cursor einer Datenbankverbindung
    
    Ausgabebeispiel:
    actions = {1: [51.37443166667289, 7.6975660000324595], 2: [51.380709, 7.671346]}
    
    """
    
    actions = defaultdict(list)

    try:
        # SQL-Abfrage
        cursor.execute("SELECT id, container_id, AsText(tracked_location) FROM actions")
        
        # Iteration durch das Ergebnis der SQL-Abfrage
        for ident, container_id, tracked_location in cursor:
            # Erhalte Koordianten der aktuellen Action in einer Liste
            location = get_floats(tracked_location)
            # Wenn die Action gültig ist, füge sie dem Dictionary zum entsprechenden Behälter hinzu
            if len(location) < 2:
                print('ungültiger Action-Datensatz. Action-ID: ' + str(ident))
            else:
                actions[container_id].append(location)

        # Bestimme aus den Actions einen zentralen Punkt für jeden Behälter
        for key, value, in actions.items():
            if len(value) < 2:
                actions[key] = value[0]
            else:
                actions[key] = calculate_center(value)
                
    except mariadb.Error as error:
        print("Error: {}".format(error))
        
    return actions


In [4]:
def get_coordinates(cursor):
    """
    Gibt eine Liste zurück, in der die Breiten- und Längengrade mit folgender Struktur gespeichert werden:
    Für jeden Container gibt es ein Tupel, in dem zwei Positionen gespeichert werden.
    1. Position ist die erwartete, wo der Behälter stehen sollte.
    2. Position ist die tatsächliche, wo sie laut GPS geortet wurde.
    Für jede Position gibt es ein Tupel, in dem Breiten- und Längengrad gespeichert werden.
    
    Eingabe: cursor einer Datenbankverbindung
    
    Ausgabebeispiel:
    coordinates = [((51.374569, 7.697542), (51.37443166667289, 7.6975660000324595)), ((51.374549, 7.697657), (51.380709, 7.671346))]
    
    """
    
    coordinates = list()
    actions = get_actions(cursor)
    
    try:
        # SQL-Abfrage
        cursor.execute("SELECT id, AsText(registered_location) FROM containers")
        
        # Iteration durch das Ergebnis der SQL-Abfrage
        for ident, registered_location in cursor:
            # Erhalte erwartete Koordianten des aktuellen Behälters in einer Liste
            registered = get_floats(registered_location)
            # Erhalte getrackte Koordianten des aktuellen Behälters in einer Liste
            tracked = actions[ident]
            # Wenn die Koordianten gültig sind, füge sie der Liste hinzu
            if len(registered) < 2 or len(tracked) < 2:
                print('ungültiger Datensatz oder es liegen keine Tracking-Infomationen für diesen Behäter vor. Behälter-ID: ' + str(ident))
            else:
                coordinates.append(((registered[0], registered[1]), (tracked[0], tracked[1])))
                
    except mariadb.Error as error:
        print("Error: {}".format(error))
        
    return coordinates
    

In [5]:
import matplotlib.pyplot as plt

def get_figure(coordinates):
    """
    Gibt eine Matplotlib-Figure zurück, auf der die übergebenen Behälterkoordinaten dargestellt werden sollen.
    
    Hinzufügen der Behälterpositionen
    Wenn der Abstand der Behälter größer als 100 Fuß / 30,48 Meter ist, werden sie rot dargestellt
    Wenn der Abstand der Behälter kleiner als 100 Fuß / 30,48 Meter ist, werden sie grün dargestellt
    
    Eingabebeispiel:
    coordinates = [((51.374569, 7.697542), (51.37443166667289, 7.6975660000324595)), ((51.374549, 7.697657), (51.380709, 7.671346))]
    
    Ausgabe: Matplotlib-Figure
    
    """
    
    fig = plt.figure()

    for container in coordinates:
        lats = [container[0][0], container[1][0]]
        lons = [container[0][1], container[1][1]]
        if calculate_distance(container[0], container[1]) > 0.0348:
            plt.plot(lons, lats, 'r') # rote Linie
            plt.plot(lons, lats, 'rs') # rotes Kästchen
        else:
            plt.plot(lons, lats, 'g') # grüne Linie
            plt.plot(lons, lats, 'gs') # grünes Kästchen
            
    return fig

In [7]:
import mysql.connector as mariadb
import mplleaflet

#Aufbauen der Datenbankverbindung
mariadb_connection = mariadb.connect(user='root', database='container')
cursor = mariadb_connection.cursor()

coordinates = get_coordinates(cursor)
                              
# Erzeugen der Matplotlib-Figure, die auf der OSM projeziert wird
mplleaflet.show(fig=get_figure(coordinates)) # Darstellung in neuem Tab
#mplleaflet.display(fig=get_figure(coordinates)) # Darstellung im Notebook

ungültiger Datensatz oder es liegen keine Tracking-Infomationen für diesen Behäter vor. Behälter-ID: 3
ungültiger Datensatz oder es liegen keine Tracking-Infomationen für diesen Behäter vor. Behälter-ID: 4
